# 1. Import Libraries

In [ ]:
import pandas as pd                 

: 

In [82]:
import numpy as np

In [19]:
import seaborn as sns

ImportError: DLL load failed while importing _c_internal_utils: The specified module could not be found.

In [20]:
import matplotlib.pyplot as plt


ImportError: DLL load failed while importing _c_internal_utils: The specified module could not be found.

In [83]:
from sklearn .ensemble import IsolationForest


In [84]:
import eda_helper_functions

# 2. Read the Training Data

In [85]:
file_path = r"C:\Users\msved\OneDrive\Documents\Campus X\FLIGHT PREDICTION USING AWS SAGEMAKER\data\train.csv"
train = pd.read_csv(file_path)
train

,date_of_journey,airlines,airlines_code,class,departure,total_stops,arrival,duration_in_minutes,source,destination,price_fare
0,2024-04-01,Vistara,UK-855,Premium Economy,00:26:00,1.0,07:27:00,334.8,Delhi,Cochin,23238
1,2024-03-26,Air India,AI-443,Economy,12:19:00,1.0,05:17:00,304.8,Kolkata,Banglore,8516
2,2024-12-09,Air India,AI-815,Business,10:46:00,1.0,18:57:00,355.2,Delhi,Cochin,53164
3,2024-09-08,Go First,G8-2519,Economy,21:06:00,0.0,13:02:00,850.2,Banglore,Delhi,6873
4,2024-05-04,Vistara,UK-929,Business,18:15:00,1.0,02:34:00,535.2,Kolkata,Banglore,22297
...,...,...,...,...,...,...,...,...,...,...,...
6831,2024-08-05,Vistara,UK-969,Business,00:44:00,0.0,17:15:00,1485.0,Banglore,Delhi,30249
6832,2024-07-27,Vistara,UK-995,Business,21:39:00,1.0,09:48:00,135.0,Delhi,Cochin,41528
6833,2024-08-14,Air India,AI-887,Business,09:21:00,0.0,07:38:00,135.0,Banglore,Delhi,25612
6834,2024-02-21,Indigo,6E-6475,Economy,07:53:00,1.0,17:31:00,589.8,Delhi,Cochin,6672


## 2.1 Data type Fixing


In [ ]:
train.dtypes

In [ ]:
train = train.assign(**{
    
    col : pd.to_datetime(train.loc[:,col], format="mixed")
    for col in ['date_of_journey','departure','arrival']
    
})

In [ ]:
train.dtypes

# 3. Hight level data summary

In [ ]:
train.info()

In [ ]:
train.describe(include="number")

In [ ]:
train.describe(include = "O")

In [ ]:
# changing stops data type to object from float 
(
train
    .assign(total_stops = train["total_stops"].astype(object))
    .describe(include = 'O')
)

In [ ]:
train['class'].unique()

## 3.1 Observation
- Vistara is occuring 3164 times as airline in 6836 observations.
- UK-819 is the common airline code, occuring 144 times. 
- Also, chances of delhi as source and coachin as destination is around 25%.

# 4. Analysis of outliers


In [ ]:
from sklearn .ensemble import IsolationForest


In [ ]:
forest = IsolationForest(n_estimators = 10 , random_state = 50)

In [ ]:
(
train
    .assign(
    outlier  = forest.fit_predict(
        train
            .drop(columns="price_fare")
            .select_dtypes(include="number")
                                 )
    ).query("outlier==-1")
)

# 5. Pair Plot

In [ ]:
eda_helper_functions.pair_plots(train)

# 6. Correlation Analysis

In [ ]:
eda_helper_functions.correlation_heatmap(train)

In [ ]:
eda_helper_functions.cramersV_heatmap(train)

# 7. Analysis in detail

In [ ]:
train.columns


## 7.1 airline category

In [ ]:
train.airlines


In [ ]:
eda_helper_functions.cat_summary(train,"airlines")
#  cat_summary means category summary and if we do seperately, then normalize=True will give the numbers in % format.

## Univariate Analysis of airline:

In [ ]:
eda_helper_functions.cat_univar_plots(train, "airlines",show_wordcloud=False,k=4)
# k = 4 means it will select top 4 frequencies and combine rest of the categories into 1

## Bi-variate Analysis between airline and price

In [ ]:
eda_helper_functions.num_cat_bivar_plots(
data=train,
num_var="price_fare",
cat_var="airlines"
)

- From the above graph, it is observed that airlines and price fare is correlated and from box plot.

In [ ]:
eda_helper_functions.num_cat_hyp_testing(train, "price_fare", "airlines")


In [ ]:
airline_grid = sns.FacetGrid(
data=train,
col="destination",
col_wrap=3,
height=4,
aspect=1.2,
sharey=True
)

airline_grid.map(sns.barplot, "airlines", "price_fare", order=train.airlines.unique())

for ax in airline_grid.axes[3:]:
    eda_helper_functions.rotate_xlabels(ax)

## 8.2 date_of_journey

In [ ]:
train.date_of_journey


In [ ]:
eda_helper_functions.dt_univar_plots(
data=train,
var="date_of_journey",
target="price_fare"
)

In [ ]:
(
    train
    .groupby(pd.Grouper(key="date_of_journey", freq="M"))
    .price_fare.mean()
    .to_frame()
    .set_axis(["March", "April", "May", "June", "July", "August", "September", "October", "November", "December", "January", "February"], axis=0)  # Update axis labels with all months
    .rename_axis(index="Month")
    .plot(
        kind="bar",
        figsize=(10, 6),
        edgecolor="black"
    )
)

ax = plt.gca()
eda_helper_functions.rotate_xlabels(ax)


In [ ]:

(
	train
	.groupby([pd.Grouper(key="date_of_journey", freq="M"), "source"])
	.price_fare.mean()
	.unstack(fill_value=0)
	.set_axis(["March", "April", "May", "June", "July", "August", "September", "October", "November", "December", "January", "February"], axis=0)
	.rename_axis(index="Month")
	.plot(
		kind="bar",
		figsize=(12, 4),
		edgecolor="green"
	)
)

ax = plt.gca()

ax.legend(
	loc="upper left",
	bbox_to_anchor=(1, 1)
)

eda_helper_functions.rotate_xlabels(ax)

## 8.3 Departure

In [ ]:
train.departure

In [ ]:
eda_helper_functions.dt_univar_plots(train, "departure", "price_fare")


## 8.4 Duration

In [ ]:
train.duration_in_minutes

In [ ]:
eda_helper_functions.num_summary(train, "duration_in_minutes")


In [ ]:
train.duration_in_minutes.quantile([0.25, 0.5, 0.75])


In [ ]:
eda_helper_functions.num_univar_plots(train, "duration_in_minutes")


In [ ]:
eda_helper_functions.num_univar_plots(train, "duration_in_minutes")

In [ ]:
eda_helper_functions.num_bivar_plots(
	train,
	"duration_in_minutes",
	"price_fare",
	hexbin_kwargs=dict(
		yscale="log",
		gridsize=20,
		cmap="Blues"
	)
)

In [ ]:
eda_helper_functions.num_num_hyp_testing(train, "price_fare", "duration_in_minutes")


In [ ]:
duration_grid = sns.FacetGrid(
	data=train,
	col="source",
	hue="total_stops",
	sharey=True,
    height = 3
)

duration_grid.map(sns.scatterplot, "duration_in_minutes", "price_fare")

duration_grid.add_legend()

In [ ]:
eda_helper_functions.get_iqr_outliers(train, "duration_in_minutes")


# 9. Automated EDA

In [80]:
%pip install ydata-profiling --user

from ydata_profiling import ProfileReport
report = ProfileReport(train)
report.to_file(output_file="output.html")

Note: you may need to restart the kernel to use updated packages.


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 79.69it/s]
